# Recommendation Systems - Lab

## Introduction

Now that you've gotten an introduction to collaborative filtering and recommendation systems, it's time to put your skills to test and attempt to build a recommendation system for a real world dataset! For this exercise, you'll be using a dataset regarding the book reviews on the Amazon marketplace. While the previous lesson focused on user-based recommendation systems, you'll apply a parallel process for an item-based recommendation system to recommend similar books at the bottom of the product page.

## Objectives

You will be able to:
* Implement a recommendation system on a real world dataset

## Load the Dataset

To start, load the dataset stored in the file `'books_data.edgelist'`.

In [1]:
import pandas as pd

books = pd.read_csv("books_data.edgelist", 
                    names=['source', 'target', 'weight'],
                   delimiter=' ')

books.head()

,source,target,weight
0,0827229534,0804215715,0.7
1,0827229534,156101074X,0.5
2,0827229534,0687023955,0.8
3,0827229534,0687074231,0.8
4,0827229534,082721619X,0.7


## Load the MetaData

Next, load the metadata associated with each of the books being reviewed. The metadata is stored in the file `'books_meta.txt'`.

In [3]:
books_meta = pd.read_csv("books_meta.txt", sep='\t')

books_meta.head()


,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,clergi sermon subject religion preach spiritu ...,Book,396585,2,5.0,8,0.80
1,2,0738700797,Candlemas: Feast of Flames,subject witchcraft earth religion spiritu base...,Book,168596,12,4.5,9,0.85
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,general hobbi subject craft home garden book,Book,1270652,1,5.0,0,0.00
3,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,spiritu translat commentari christian book gui...,Book,631289,1,4.0,6,0.79
4,5,1577943082,Prayers That Avail Much for Business: Executive,subject religion spiritu busi christian live w...,Book,455160,0,0.0,4,1.00


## Create an Item Matrix

This is essentially the same as the user based matrix you saw constructed in the previous lesson, but for items versus other items. From this, you'll then select the most similar items in order to produce a recommendation suitable for the bottom of a product page.

## Select Books to Test Your Recommender On

Select a small subset of books that you are interested in generating recommendations for. 

In [7]:
bird = books_meta[books_meta.Title.str.contains('bird')]
bird

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
126,148,0812550749,Firebird,general subject merced author l z paperback fa...,Book,77008,42,4.0,4,1.00
1452,2068,1555660126,Hummers: Hummingbirds of North America (Pocket...,general subject outdoor natur bird ecolog book...,Book,700069,3,5.0,5,0.77
3143,4400,1570612218,River-Walking Songbirds & Singing Coyotes: An ...,biolog general subject unit outdoor natur stat...,Book,742835,3,5.0,4,0.90
5682,8001,1580070531,Vought F4U Corsair: WarbirdTech Volume 4,militari subject aviat pictori book scienc his...,Book,188421,1,3.0,4,0.90
9192,13155,1583880283,Pontiac Firebird 1967-2000 Photo History,nonfict general subject pictori automot transp...,Book,450624,2,3.5,5,0.90
9988,14326,0123569559,Hummingbirds of North America: A Photographic ...,art general subject camera essay outdoor natur...,Book,610360,1,5.0,4,1.00
10105,14492,0816730520,The Hummingbird King: A Guatemalan Legend (Leg...,folk general subject literatur stori fairi tal...,Book,380276,3,4.5,6,0.81
12365,17591,1565105753,Readings on to Kill a Mockingbird (Greenhaven ...,specialti unknown book store jp,Book,83715,1,5.0,0,0.00
14508,20552,0875968619,Attracting Hummingbirds and Butterflies to You...,biolog general insect subject home techniqu ou...,Book,119277,4,5.0,5,0.48
15055,21384,0876669577,All About Lovebirds,general subject care pet home garden anim book,Book,1516976,0,0.0,0,0.00


## Generate Recommendations for a Few Books of Choice

Now that you have the preprocessed and transformed the data, it's time to employ collaborative filtering to generate recommendations! Be sure to print the book name that you are generating recommendations for as well as the name of the books being recommended.

In [ ]:
rec_dict = {}
for row in bird.index:
    book_id = bird.ASIN[row]
    book_name = id_name_dict[book_id]
    

## Summary

Well done! In this lab, you effectively created a recommendation system for a real world dataset!